In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False

의사 결정 트리(Decision Tree)  
의사 결정 트리에서는 테스트를 위쪽에 놓고 해당 테스트로 발생 가능한 결과를 아래쪽에 놓는다. 의사 결정 트리의 결과는 또 다른 테스트가 될 수 있다. 즉, 상위 테스트로 판별하지 못한 데이터는 또 다른 테스트인 자식 테스트를 사용해 분류가 가능하다.

엔트로피(Entropy)  
테스트 성능을 평가하기 위해 엔트로피라는 개념을 사용한다.  
엔트로피는 불순도 정도를 측정하며, 낮을수록 좋가. 이때, 불순도는 노드에 서로 다른 데이터가 얼마나 섞여있는지를 의미한다. 불순도가 낮을수록 데이터가 섞여있지 않는다는 것을 의미한다. 엔트로피는 아래와 같은 식을 따른다.

$$entropy(d) = -\sum P(x)logP(x) = -\sum_{i=1}^k P(i|d)log_2(p(i|d))$$

위 식에서 $P(i|d)$는 노드 $d$가 주어질 때 $i$ 클래스에 속할 확률이다. 이진 분류 문제는 $k=2$이며, 이를 적용하면 아래와 같이 표현할 수 있다.

$$entropy(d) = -\frac{P}{T}log_2\frac{P}{T} - \frac{N}{T}log_2\frac{N}{T}$$

위 식에서 $T$는 노드 속 전체 데이터 개수, $P$는 Positive 데이터 개수, $N$은 Nagative 데이터 개수를 의미한다.

지니 계수(Gini Index)  
지니 계수는 엔트로피와 함께 불순도를 측정하는 또 다른 방법이다.  
지니 계수는 데이터셋에서 랜덤으로 삭제한 데이터에 임의로 라벨을 정했을 때 틀릴 확률을 의미한다. 만일 해당 데이터셋의 데이터가 모두 동일하게 라벨링되어 있다면 지니 계수는 0을 가지게 되며 이는 불순도가 0임을 의미한다.

의사 결정 트리 알고리즘을 활용해 와인 종류를 구분한다.

In [2]:
# 데이터 불러오기
from sklearn import datasets # 사이킷런이 제공하는 데이터셋을 사용하기 위해 import 한다.
raw_wine = datasets.load_wine() # 와인 데이터를 불러온다.

In [3]:
# 피쳐, 타겟 데이터 저장
X = raw_wine.data # 피쳐 데이터를 저장한다.
y = raw_wine.target # 타겟 데이터를 저장한다.
X.shape

(178, 13)

In [4]:
# 트레이닝, 테스트 데이터 분할
from sklearn.model_selection import train_test_split # 트레이닝, 테스트 데이터 분할을 위해 import 한다.
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0)
print(X_tn.shape, y_tn.shape, X_te.shape, y_te.shape) # 트레이닝 데이터와 테스트 데이터로 분할한다.

(133, 13) (133,) (45, 13) (45,)


In [5]:
# 데이터 표준화
from sklearn.preprocessing import StandardScaler # 데이터 표준화를 위해 import 한다.
std_scale = StandardScaler() # 표준화 스케일러 객체를 만든다.
# 표준화는 트레이닝 데이터를 기반으로 실행하므로 트레이닝 피쳐 데이터 X_tn을 표준화 스케일러에 적합시킨다.
X_tn_std = std_scale.fit_transform(X_tn) # 트레이닝 데이터를 적합 후 표준화 한다.
X_te_std = std_scale.transform(X_te) # 테스트 데이터를 표준화 한다.

In [8]:
# 모델 생성 후 데이터 학습
from sklearn.tree import DecisionTreeClassifier # 의사 결정 트리 알고리즘을 사용하기 위해 import 한다.
clf_tree = DecisionTreeClassifier(random_state=0) # 의사 결정 트리 모델을 만든다.
# 표준화된 트레이닝 피쳐 데이터 X_tn_std와 트레이닝 타겟 데이터 y_tn를 모델에 넣어서 선형 회귀 알고리즘을 학습시킨다.
clf_tree.fit(X_tn_std, y_tn)

DecisionTreeClassifier(random_state=0)

In [9]:
# 테스트 데이터 예측
# 표준화된 테스트 데이터 X_te_std로 예측한다.
pred_tree = clf_tree.predict(X_te_std)
print(pred_tree)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 2 0 1 0 1 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 1 0 1 1 1]


In [10]:
# 정확도 평가
from sklearn.metrics import accuracy_score # 모델의 정확도를 평가하기 위해 import 한다.
# accuracy_score() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_tree)를 넘겨 정확도를 계산한다.
accuracy = accuracy_score(y_te, pred_tree)
print(accuracy)

0.9333333333333333


In [11]:
# 정밀도 평가
from sklearn.metrics import precision_score # 모델의 정밀도를 평가하기 위해 import 한다.
# precision_score() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_tree)를 넘겨 정밀도를 계산한다.
precision = precision_score(y_te, pred_tree, average=None)
print(precision)

[1.         0.90909091 0.88888889]


In [12]:
# 재현율 평가
from sklearn.metrics import recall_score # 모델의 재현율를 평가하기 위해 import 한다.
# recall_score() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_tree)를 넘겨 재현율를 계산한다.
recall = recall_score(y_te, pred_tree, average=None)
print(recall)

[0.875      0.95238095 1.        ]


In [13]:
# f1 score 평가
from sklearn.metrics import f1_score # 모델의 f1 score를 평가하기 위해 import 한다.
# f1_score() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_tree)를 넘겨 f1 score를 계산한다.
f1 = f1_score(y_te, pred_tree, average=None)
print(f1)

[0.93333333 0.93023256 0.94117647]


In [14]:
# 혼돈 행렬 확인
from sklearn.metrics import confusion_matrix # 혼돈 행렬을 만들기 위해 import 한다.
# confusion_matrix() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_tree)를 넘겨 혼돈 행렬을 만든다.
conf_matrix = confusion_matrix(y_te, pred_tree)
print(conf_matrix)

[[14  2  0]
 [ 0 20  1]
 [ 0  0  8]]


In [15]:
# 분류 리포트 확인
from sklearn.metrics import classification_report # 분류 리포트를 출력하기 위해 import 한다.
# classification_report() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_tree)를 넘겨 분류 리포트를 만든다.
target_names = raw_wine.target_names
class_report = classification_report(y_te, pred_tree, target_names=target_names)
print(class_report)

              precision    recall  f1-score   support

     class_0       1.00      0.88      0.93        16
     class_1       0.91      0.95      0.93        21
     class_2       0.89      1.00      0.94         8

    accuracy                           0.93        45
   macro avg       0.93      0.94      0.93        45
weighted avg       0.94      0.93      0.93        45

